人脸检测演示程序

作者：谢文伟

邮件：jim.xie.cn@outlook.com

主页：https://github.com/jim-xie-cn/ai-cv

# 导入库

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage import data,color,feature,transform
from sklearn.feature_extraction.image import PatchExtractor
from sklearn.datasets import fetch_lfw_people
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
import dlib
from mtcnn.mtcnn import MTCNN

# 创建人脸二分类器

In [ ]:
#从lfw数据集上获取人脸图片，做为训练的正例
faces = fetch_lfw_people()
samples_with_face = faces.images
print("正例样本数量: ",samples_with_face.shape[0])
print("图像尺寸(WxH): %dx%d"%(samples_with_face.shape[2],samples_with_face.shape[1]))
#显示正例样本的前96张图片
plt.style.use({'figure.figsize':(8, 6)})
fig, ax = plt.subplots(8, 12)
for i, axi in enumerate(ax.flat):
    axi.imshow(samples_with_face[ i], cmap='gray',interpolation='none')
    axi.axis('off')

In [ ]:
#使用sickit-image内置（不含头像）的图像作为负例样本
skimg_without_face = ['text','coins','moon','page','clock',
                      'immunohistochemistry','chelsea','coffee', 
                      'hubble_deep_field','checkerboard']
images = [color.rgb2gray(getattr(data, name)()) for name in skimg_without_face]
#由于内置图片太少，使用PatchExtractor在原图基础上生成更多图片作为负例样本
def generate_patches(img, count, scale=1.0):
    extractor = PatchExtractor(patch_size=samples_with_face[0].shape,max_patches=count)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch,samples_with_face[0].shape) for patch in patches])
    return patches
samples_without_face = np.array([])
for im in images :
    for scale in [0.5, 0.75,1.0, 1.5,2.0]:
        batch = generate_patches(im, 1000, scale)
        if not samples_without_face.any():
            samples_without_face = np.array(batch)
        else:
            samples_without_face = np.vstack((samples_without_face,batch))
print("负例样本数量: ",samples_without_face.shape[0])
print("图像尺寸(WxH): %dx%d"%(samples_without_face.shape[2],samples_without_face.shape[1]))
#显示负例样本中96张图片
plt.style.use({'figure.figsize':(8, 6)})
fig, ax = plt.subplots(8, 12)
for i, axi in enumerate(ax.flat):
    axi.imshow(samples_without_face[500 * i], cmap='gray')
    axi.axis('off')

生成训练所需的数据集

In [ ]:
#将正例和负例样本合并到all_samples数组中
all_samples = np.vstack((samples_with_face,samples_without_face))
X_train = []
#提取HOG特征并生成数据集
for im in all_samples:
    X_train.append(feature.hog(im))
X_train = np.array(X_train)
#带有人脸图像的标签设为1,不带有人脸图像的标签设为0
Y_train = np.zeros(all_samples.shape[0])
Y_train[:samples_with_face.shape[0]] = 1
print("数据集中的样本数量",X_train.shape[0])

使用SVM进行训练，得到人脸二分类模型

In [ ]:
#选择线性SVM(LinearSVC)作为分类模型
svc = LinearSVC()
#使用网格搜索寻找最优参数
model_grid = GridSearchCV(svc,{'C': [0.5, 1.0, 1.5, 2.0]})
print("正在训练，需要时间较长，请稍后...")
model_grid.fit(X_train, Y_train)
print("最优模型得分：",model_grid.best_score_,"最优模型参数：",model_grid.best_params_)
model = model_grid.best_estimator_

# 人脸初检

定义滑动窗口函数

In [ ]:
#根据指定的步长和窗口大小，遍历图像，取出HOG特征
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize[1]):
        for x in range(0, image.shape[1], stepSize[0]):
            region = image[y:y+windowSize[0], x:x+windowSize[1]]
            region = transform.resize(region, windowSize)
            region_hog = feature.hog(region) 
            yield ((x, y, x + windowSize[1],y+windowSize[0]),region,region_hog)

In [ ]:
#读取图像，转换为灰度图并调整至统一的大小
img = cv2.imread('./images/boy.jpg')
gray = color.rgb2gray(img)
gray = transform.resize(gray,(120,100))
#使用滑动窗口，提取图像中不同区域的HOG特征
rects, region_img,regions_hog = zip(*sliding_window(gray,stepSize=(2,2), windowSize=(62,47)))
#使用训练好的SVM模型，对每个区域进行识别
predict_regions = model.predict(regions_hog)
face_index = np.argwhere(predict_regions == 1)
print("待检图像尺寸:%dx%d"%(gray.shape[0],gray.shape[1]),"滑动窗口尺寸:47x62","移动步长:2x2")
print("待检区域（共%d个）"%len(rects))

可视化初检结果

In [ ]:
#显示滑动窗口扫描的区域
plt.style.use({'figure.figsize':(8, 6)})
fig, ax = plt.subplots(8, 12)
for i, axi in enumerate(ax.flat):
    axi.imshow(region_img[i], cmap='gray')
    axi.axis('off')
plt.show()
#显示可能存在人脸的区域
print("检测到的人脸区域（共%d个）"%len(face_index))
fig, ax = plt.subplots()
img1 = cv2.imread('./images/boy.jpg')
img1= cv2.cvtColor(img1.copy(),cv2.COLOR_BGR2RGB)
img1 = transform.resize(img1,(120,100))
ax.imshow(img1)
for i in np.squeeze(face_index):
    (x1,y1,x2,y2) = rects[i]
    ax.add_patch(plt.Rectangle((x1,y1),x2-x1,y2-y1,edgecolor='red',lw=1,facecolor='none'))
    ax.axis('off')

In [ ]:
img = cv2.imread('./images/boy.jpg')
img= cv2.cvtColor(img.copy(),cv2.COLOR_BGR2RGB)
plt.style.use({'figure.figsize':(16, 12)})
fig, ax = plt.subplots(1, 3)
for i, axi in enumerate(ax.flat):
    axi.imshow(img, cmap='gray')
    axi.axis('off')
    axi.set_title("%dx%d"%(img.shape[1],img.shape[0]))
    img = cv2.pyrDown(img)

# 初检结果修正

In [ ]:
 def nms(boxes, threshold):
    if boxes.size == 0:
        return np.empty((0, 3))
    x1,y1,x2,y2,s = boxes[:,0],boxes[:,1],boxes[:,2],boxes[:,3],boxes[:,4]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    sorted_s = np.argsort(s)
    pick = np.zeros_like(s, dtype=np.int16)
    counter = 0
    while sorted_s.size > 0:
        i = sorted_s[-1]
        pick[counter] = i
        counter += 1
        idx = sorted_s[0:-1]
        xx1,yy1 = np.maximum(x1[i], x1[idx]),np.maximum(y1[i], y1[idx]),
        xx2,yy2 = np.minimum(x2[i], x2[idx]),np.minimum(y2[i], y2[idx])
        w,h = np.maximum(0.0, xx2 - xx1 + 1),np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        o = inter / (area[i] + area[idx] - inter)
        sorted_s = sorted_s[np.where(o <= threshold)]
    pick = pick[0:counter]
    return pick

使用NMS去掉重复的人脸区域

In [ ]:
#取出模型分类的分值，该分值表示该区域存在人脸的概率
scores= model.decision_function(regions_hog) 
#使用NMS算法去重，阈值threshold设置为0.9
boxes_list = []
for i in np.squeeze(face_index):
    (x1,y1,x2,y2) = rects[i]
    s = scores[i]
    if s > 0.9:
        boxes_list.append((x1,y1,x2,y2,s))
boxes_list = np.squeeze(np.array(boxes_list))
best_boxes = nms(boxes_list,0.1)
#显示去重后的结果
fig, ax = plt.subplots()
img1 = cv2.imread('./images/boy.jpg')
img1= cv2.cvtColor(img1.copy(),cv2.COLOR_BGR2RGB)
img1 = transform.resize(img1,(120,100))
ax.imshow(img1)
for i in best_boxes:
    (x1,y1,x2,y2,s) = boxes_list[i]
    ax.add_patch(plt.Rectangle((x1,y1),x2-x1,y2-y1,edgecolor='red',lw=1,facecolor='none'))
    ax.axis('off')
plt.show()

# 10.3.4  使用开源模型实现人脸检测

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
img = cv2.imread('./images/boy.jpg')
img= cv2.cvtColor(img.copy(),cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray)
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
plt.imshow(img.squeeze())
plt.axis('off') 

In [ ]:
image = cv2.imread('./images/boy.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
hogFaceDetector = dlib.get_frontal_face_detector()
faces = hogFaceDetector(gray, 1)
for (i, rect) in enumerate(faces):
    x1,y1,x2,y2 = rect.left(),rect.top(),rect.right(),rect.bottom()
    cv2.rectangle(img, (x1, y1), (x2,y2), (255, 0, 0), 2)
plt.imshow(img.squeeze())
plt.axis('off') 

In [ ]:
detector = MTCNN()
img = cv2.imread('./images/boy.jpg')
img= cv2.cvtColor(image.copy(),cv2.COLOR_BGR2RGB)
faces = detector.detect_faces(img)# result
for result in faces:
    x, y, w, h = result['box']
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
plt.imshow(img.squeeze(),cmap='gray')
plt.axis('off') 